# ☕️ Coffee Machine Koans
**Goal:** Build a complete coffee ordering system from menu to history.

## 🛠️ Setup & Utilities
Run this helper cell before the tests to load the validation helpers.

In [ ]:
def validate_test_case(test_pair, error_template, error_list):
    actual_result, expected_value = test_pair
    if actual_result != expected_value:
        msg = error_template.format(actual=actual_result, expected=expected_value)
        error_list.append(f"❌ {msg}")

def log_errors(errors):
    if errors:
        for err in errors:
            print(err)
    else:
        print("✅ All Tests Passed!")


## 1. Menu Module
Structure the menu so other functions can reference it.

### Key Concepts:
* Modular design keeps the menu a single source of truth.
* Print formatting makes CLI menus easier to read.
* Functions encapsulate menu rendering logic.

Concepts: `31-maquina-cafe.md`

**Task:** Create `show_menu()` that returns the formatted menu string.


In [ ]:
MENU_OPTIONS = {
    '1': ('Espresso', 2.5),
    '2': ('Capuchino', 3.0),
    '3': ('Latte', 3.5)
}

def show_menu():
    lines = ['Available Coffees:']
    # TODO: Iterate over MENU_OPTIONS and build the menu lines
    for key, (name, price) in MENU_OPTIONS.items():
        lines.append(f"{key}. {name} - ${price:.2f}")
    return '
'.join(lines)


In [ ]:
# 🧪 TEST BLOCK
errors = []
expected = '
'.join([
    'Available Coffees:',
    '1. Espresso - $2.50',
    '2. Capuchino - $3.00',
    '3. Latte - $3.50'
])
validate_test_case((show_menu(), expected), "Menu formatting mismatch.", errors)
log_errors(errors)


## 2. Order Processing
Validate customer input and read from dictionary options.

### Key Concepts:
* Dictionaries map option keys to coffee tuples (name, price).
* Input validation prevents invalid states.
* Lookup uses `.get()` plus branching logic for clarity.

Concepts: `31-maquina-cafe.md`

**Task:** Implement `order_coffee()` that returns a descriptive message or an invalid warning.


In [ ]:
def order_coffee(option, customer_name):
    choice = MENU_OPTIONS.get(option)
    # TODO: Reject invalid options early
    if choice is None:
        return 'Invalid Option'
    coffee_name, price = choice
    # TODO: Return a message summarizing the order
    return f"{customer_name} ordered {coffee_name} for ${price:.2f}"


In [ ]:
# 🧪 TEST BLOCK
errors = []
validate_test_case((order_coffee('1', 'Alex'), 'Alex ordered Espresso for $2.50'), "Order processing failed for Espresso.", errors)
validate_test_case((order_coffee('2', 'Bianca'), 'Bianca ordered Capuchino for $3.00'), "Order processing failed for Capuchino.", errors)
validate_test_case((order_coffee('9', 'Zoe'), 'Invalid Option'), "Order processing should reject invalid choices.", errors)
log_errors(errors)


## 3. File Persistence
Save orders without losing previous entries.

### Key Concepts:
* Use `'a'` mode to append without erasing history.
* Encoding (e.g., `'utf-8'`) keeps data portable.
* `with` statements close files even if errors occur.

Concepts: `32-escritura-lectura.md`

**Task:** Append order lines to the history file.


In [ ]:
def save_order(order_details, filename='orders.txt'):
    # TODO: Append the order_details to the specified file
    with open(filename, 'a', encoding='utf-8') as writer:
        writer.write(order_details + '
')


In [ ]:
import os, tempfile

# 🧪 TEST BLOCK
errors = []
history_path = os.path.join(tempfile.gettempdir(), 'coffee_orders_test.txt')
if os.path.exists(history_path):
    os.remove(history_path)
save_order('Alex ordered Espresso for $2.50', history_path)
with open(history_path, 'r', encoding='utf-8') as reader:
    content = reader.read().strip()
validate_test_case((content, 'Alex ordered Espresso for $2.50'), "Persistence: content mismatch.", errors)
if os.path.exists(history_path):
    os.remove(history_path)
log_errors(errors)


## 4. History Reader
Read the saved orders with graceful fallback.

### Key Concepts:
* Use `try` / `except FileNotFoundError` when the log may not exist.
* `readlines()` and `enumerate()` turn file rows into numbered records.
* Return structured responses instead of letting the error bubble up.

Concepts: `31-maquina-cafe.md`

**Task:** Implement `view_history()` so callers always receive a list.


In [ ]:
def view_history(filename='orders.txt'):
    try:
        with open(filename, 'r', encoding='utf-8') as reader:
            return [f"{idx + 1}. {line.strip()}" for idx, line in enumerate(reader) if line.strip()]
    except FileNotFoundError:
        return []


In [ ]:
import os, tempfile

# 🧪 TEST BLOCK
errors = []
history_path = os.path.join(tempfile.gettempdir(), 'coffee_history_test.txt')
with open(history_path, 'w', encoding='utf-8') as writer:
    writer.write('Anna ordered Latte for $3.50
')
    writer.write('Ben ordered Capuchino for $3.00
')
validate_test_case((view_history(history_path), ['1. Anna ordered Latte for $3.50', '2. Ben ordered Capuchino for $3.00']), "History reader failed.", errors)
if os.path.exists(history_path):
    os.remove(history_path)
validate_test_case((view_history(history_path), []), "History should return [] when the file is missing.", errors)
log_errors(errors)


## 5. Main Loop Integration
Wire the menu, ordering, persistence, and history into one flow.

### Key Concepts:
* `while True` loops run until explicitly broken.
* Use `break` to exit when no more commands remain.
* Guard scripts with `if __name__ == '__main__'` before publishing.

Concepts: `31-maquina-cafe.md`

**Task:** Implement `run_coffee_shop()` that iterates through actions and persists each order.


In [ ]:
def run_coffee_shop(actions, history_file='orders.txt'):
    served = []
    index = 0
    while True:
        if index >= len(actions):
            break
        option, customer = actions[index]
        if option.lower() == 'exit':
            break
        order_text = order_coffee(option, customer)
        if order_text == 'Invalid Option':
            break
        save_order(order_text, history_file)
        served.append(order_text)
        index += 1
    return served

# Uncomment to run from a script. Not auto-executed inside the notebook.
# if __name__ == '__main__':
#     sample = [('1', 'Ava'), ('3', 'Leo'), ('exit', '')]
#     run_coffee_shop(sample)


In [ ]:
import os, tempfile

# 🧪 TEST BLOCK
errors = []
history_path = os.path.join(tempfile.gettempdir(), 'coffee_main_test.txt')
if os.path.exists(history_path):
    os.remove(history_path)
actions = [('1', 'Ava'), ('5', 'Zoe')]
served = run_coffee_shop(actions, history_path)
validate_test_case((served, ['Ava ordered Espresso for $2.50']), "Run loop should stop after invalid option.", errors)
history_lines = view_history(history_path)
validate_test_case((history_lines, ['1. Ava ordered Espresso for $2.50']), "History log mismatch after run.", errors)
if os.path.exists(history_path):
    os.remove(history_path)
log_errors(errors)
